In [17]:
import requests,json,sys,os, gspread, time, re

import numpy as np

if 'google.colab' in str(get_ipython()):
    %pip install python-docx
    %pip install kanjize
    from kanjize import int2kanji, kanji2int
    from google.colab import files,auth
    from oauth2client.client import GoogleCredentials
    outputdirectory = ''
else:
    from kanjize import int2kanji, kanji2int
    outputdirectory = './outputs/' #ローカルで実行する場合は保存ファイルのディレクトリを適当に指定
    os.makedirs(outputdirectory,exist_ok=True)
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_UNDERLINE
import pandas as pd
file_name=outputdirectory+file_name_download

In [7]:
#スプレッドシートをダウンロード
if 'google.colab' in str(get_ipython()):
    sheeturl='https://docs.google.com/spreadsheets/d/1n04e6POI04TBt-3HJUH10-T5cxhPZHcBWmFA4tSHjqE/edit#gid=288090143'
    sheeturl_csv=re.match("https://docs.google.com/spreadsheets/d/.+/",sheeturl).group(0)+"export?format=csv"
    aozoradata=pd.read_csv(sheeturl_csv)
else:
    aozoradata=pd.read_excel('青空文庫API用データ.xlsx')


In [41]:
listfile='https://raw.githubusercontent.com/dbkk/aozoraAPI_to_docx/master/titles.list'
with open(listfile, 'r') as f:
    titles = f.read().splitlines()

sub_df=aozoradata[aozoradata['作品名'].str.contains('|'.join(titles), na=False)]

s_df = sub_df.sort_values('底本名1')

In [45]:
docx_lines=[]
for index, s in s_df.iterrows():
    txt=s['初出']
    japtxt=''
    for t in txt:
        if t.isdigit():
            if t=='0':
                japtxt+='〇'
            else:
                japtxt+=int2kanji(int(t))
        else:
            japtxt+=t
    ytxt=s['底本初版発行年1'].split('（')[0]
    yjaptxt=''
    for t in ytxt:
        if t.isdigit():
            if t=='0':
                yjaptxt+='〇'
            else:
                yjaptxt+=int2kanji(int(t))
        else:
            yjaptxt+=t
    newline=s['姓名']+'　「'+s['作品名']+'」　'+s['底本名1']+'　'+s['底本出版社名1']+'　'+yjaptxt+'　（初出：'+japtxt+'）'
    print(newline)
    docx_lines.append(newline)


中谷宇吉郎　「救われた稀本」　中谷宇吉郎集　第五巻　岩波書店　二〇〇一　（初出：「物理学序説」岩波書店、一九四七（昭和二二）年四月五日）
中谷宇吉郎　「湯川秀樹さんのこと」　中谷宇吉郎集　第六巻　岩波書店　二〇〇一　（初出：「文藝春秋　第二十八巻第一号」文藝春秋新社、一九五〇（昭和二五）年一月一日）
寺田寅彦　「春六題」　寺田寅彦随筆集　第一巻　岩波文庫、岩波書店　一九四七　（初出：「新文学」一九二一（大正一〇）年四月）
寺田寅彦　「物理学と感覚」　寺田寅彦随筆集　第一巻　岩波文庫、岩波書店　一九四七　（初出：「東洋学芸雑誌」一九一七（大正六）年一一月）
寺田寅彦　「量的と質的と統計的と」　寺田寅彦随筆集　第三巻　岩波文庫、岩波書店　一九四八　（初出：「科学」一九三一（昭和六）年一〇月）
寺田寅彦　「物理学圏外の物理的現象」　寺田寅彦随筆集　第三巻　岩波文庫、岩波書店　一九四八　（初出：「理学界」一九三二（昭和七）年一月）
寺田寅彦　「備忘録」　寺田寅彦随筆集　第二巻　岩波文庫、岩波書店　一九四七　（初出：「思想」一九二七（昭和）二年九月）
寺田寅彦　「自然界の縞模様」　寺田寅彦随筆集　第四巻　岩波文庫、岩波書店　一九四八　（初出：「科学」一九三三（昭和八）年二月）
寺田寅彦　「物質群として見た動物群」　寺田寅彦随筆集　第四巻　岩波文庫、岩波書店　一九四八　（初出：「理学界」一九三三（昭和八）年四月）


In [47]:
# generate docx

file_name_download = "output.docx"

document = Document()
document.add_paragraph('引用文献')
inds=0
for nl in docx_lines:
    p = document.add_paragraph(nl)

document.save(file_name_download)

In [12]:
if 'google.colab' in str(get_ipython()):
    files.download(file_name_download)